In [37]:
import pandas as pd
import glob
import re
import unicodedata
import numpy as np
import ast 

Nesta secção, vamos juntar todos os produtos extraidos  de cada um dos sites seja em black friday ou não, vamos criar um dataset com todos os produtos e respectivas informações. Depois vamos fazer uma limpeza dos dados repetidos e por fim aplicar um modelo de agregação e categorização dos produtos.

In [38]:
file_paths = glob.glob("data/*products_extracted_prices*.csv") # with glob we can get all the files that match the pattern
dataframes = []

for file_path in file_paths:
    df = pd.read_csv(file_path)
    
    if "Black_Friday" in file_path:
        df['promocao'] = 'Black Friday'
    else:
        df['promocao'] = 'Sem Promocao'
    
    dataframes.append(df)

merged_df = pd.concat(dataframes, ignore_index=True)

merged_df.to_csv("all_prices_extracted.csv", index=False)

Abaixo vou so fazer um teste para ter a certeza que o codigo acima funcionou como pretendido.

In [39]:
file_paths = glob.glob("data/*products_extracted_prices*.csv") 

black_friday_counts = {}
not_black_friday_counts = {}

for file_path in file_paths:
    if "Black_Friday" in file_path:
        df = pd.read_csv(file_path)
        black_friday_counts[file_path] = df.shape[0]
    else:
        df = pd.read_csv(file_path)
        not_black_friday_counts[file_path] = df.shape[0]
        
total_black_friday_rows = sum(black_friday_counts.values())
print(f"Total Black Friday rows across all files: {total_black_friday_rows}")
total_black_friday_rows_on_merged_df = merged_df[merged_df['promocao'] == 'Black Friday'].shape[0]
print(f"Total Black Friday rows on merged DataFrame: {total_black_friday_rows_on_merged_df}")

total_not_black_friday_rows = sum(not_black_friday_counts.values())
print(f"Total Not Black Friday rows across all files: {total_not_black_friday_rows}")
total_not_black_friday_rows_on_merged_df = merged_df[merged_df['promocao'] == 'Sem Promocao'].shape[0]
print(f"Total Not Black Friday rows on merged DataFrame: {total_not_black_friday_rows_on_merged_df}")

Total Black Friday rows across all files: 839
Total Black Friday rows on merged DataFrame: 839
Total Not Black Friday rows across all files: 703
Total Not Black Friday rows on merged DataFrame: 703


In [40]:
merged_df = pd.read_csv("all_prices_extracted.csv")
merged_df = merged_df.drop(columns=['linkToArchive','linkToExtractedText','snippet'])

In [41]:
merged_df.isna().sum()


site              0
date              0
title            16
extractedData    55
promocao          0
dtype: int64

In [42]:
# eliminate rows with missing values
merged_df = merged_df.dropna()
merged_df.isna().sum()

site             0
date             0
title            0
extractedData    0
promocao         0
dtype: int64

In [43]:
print(f"Shape before: {merged_df.shape}")

merged_df = merged_df.drop_duplicates()

print(f"Shape after: {merged_df.shape}")


Shape before: (1476, 5)
Shape after: (967, 5)


In [44]:
def clean_extracted_data(extracted_str):
    try:
        extracted_list = ast.literal_eval(extracted_str) # é suado para converter as strings em listas
    except (ValueError, SyntaxError):
        return []  

    cleaned_prices = []
    for price in extracted_list:
        normalized_price = unicodedata.normalize("NFKD", price)
        cleaned_price = re.sub(r'[^\d,]', '', normalized_price)
        cleaned_prices.append(cleaned_price)
    
    return cleaned_prices

merged_df['extractedData'] = merged_df['extractedData'].apply(clean_extracted_data)


In [45]:
stop_words = ["Dual", "SIM", "Sim", "Staples.pt", "|", "Â", "Tecnologia", "El" ,"Corte" "Â·", "Android", '""', '"', ",", "InglÃ©s", "InglÃ©s", "Inglés", "·",  "Sabe", "mais",  "em", "Fnac.pt", "Compra","Comprar", "na", "(", "Product", ")", "Worten.pt", "Worten", "pt", "WORTEN", "-", '"', "Wi-Fi", "Cuidados",  "Ecológicos","Vida", "Sustentável", "Radio", "Popular", "PCDIGA", "(Product)"]

def clean_title(title):
    title = title.split()
    title = [word for word in title if word not in stop_words]
    title = ' '.join(title)
    return title

merged_df['title'] = merged_df['title'].apply(clean_title)

In [46]:
# quero retirar cenas do genero N.N"" ou N.N'' onde NN é um número, quero que substitua por 
def clean_title(title):
    title = re.sub(r'\d+\.\d+["\']', '', title)
    return title

merged_df['title'] = merged_df['title'].apply(clean_title)

# quero retirar ' 
def clean_title(title):
    title = re.sub(r"'", '', title)
    return title

merged_df['title'] = merged_df['title'].apply(clean_title)

In [47]:
merged_df.to_csv("all_prices_extracted_cleaned.csv", index=False)

In [48]:
df = pd.read_csv("all_prices_extracted_cleaned.csv")

df.isna().sum()

site             0
date             0
title            3
extractedData    0
promocao         0
dtype: int64

In [49]:
df = df.dropna()

In [50]:
print(f"Shape before: {df.shape}")

def remove_duplicates(df):
    '''
        Vamos remover todos os duplicados que sejam iguais nas colunas 'site', 'title', 'date' e 'promocao'
    '''
    df = df.sort_values(by=['site', 'date', 'title', 'promocao'])

    duplicates = df[df.duplicated(subset=['site', 'date', 'title', 'promocao'], keep=False)]
    print("Duplicatas encontradas: ", duplicates.shape[0])


    df = df.drop_duplicates(subset=['site', 'title', 'date', 'promocao'], keep='first')
    
    return df

# Chama a função para remover as duplicatas
df = remove_duplicates(df)

print(f"Shape after: {df.shape}")

Shape before: (964, 5)
Duplicatas encontradas:  70
Shape after: (925, 5)


In [51]:
df['extractedData'] = df['extractedData'].apply(ast.literal_eval)

def convert_to_float_list(data):
    return [float(x.replace(',', '.')) for x in data]

df['extractedData'] = df['extractedData'].apply(convert_to_float_list)

# quero ver o tipo da coluna extractedData, e o tipo do elemento da lista
print(df['extractedData'].dtype)
print(type(df['extractedData'].iloc[0][0]))

object
<class 'float'>


In [52]:
def cleaned_www(data):
    return data.split('.')[1]

df['site'] = df['site'].apply(cleaned_www)


In [53]:
df.to_csv("all_prices_extracted_cleaned.csv", index=False)